In [10]:
# _*_coding:utf-8 _*_
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [11]:
import numpy as np
from gensim import corpora, models, similarities
from pprint import pprint
import time
import jieba
import os
from six import iteritems

2016-10-23 10:58:46,298 : INFO : Pattern library is not installed, lemmatization won't be available.
2016-10-23 10:58:48,595 : INFO : 'pattern' package not found; tag filters are not available for English


In [8]:
basedir = "/home/li/corpus/news/"
dir_list = ['affairs','constellation','economic','edu','ent','fashion','game','home','house','lottery','science','sports','stock']

In [77]:
##将文本和标签分开存放，有助于训练

fw = open("news.tab","w")
fw_type = open("type.tab","w")
num = -1
for e in dir_list:
    num += 1
    indir = basedir + e + '/'
    files = os.listdir(indir)
    count = 0
    for file in files:
        if count > 10000:
            break
        count += 1            
        filepath = indir + file
        with open(filepath,'r') as fr:
            text = fr.read()
        text = text.decode("utf-8").encode("utf-8")
        seg_text = jieba.cut(text.replace("\t"," ").replace("\n"," "))
        outline = " ".join(seg_text) + "\n"
        outline = outline.encode("utf-8")
        #print outline
        fw.write(outline)
        fw.flush()
        fw_type.write(str(num) + "\n")
        fw_type.flush()
fw.close()
fw_type.flush()

In [17]:
##生成fastext的训练和测试数据集

ftrain = open("news_fasttext_train.txt","w")
ftest = open("news_fasttext_test.txt","w")

num = -1
for e in dir_list:
    num += 1
    indir = basedir + e + '/'
    files = os.listdir(indir)
    count = 0
    for file in files:
        count += 1            
        filepath = indir + file
        with open(filepath,'r') as fr:
            text = fr.read()
        text = text.decode("utf-8").encode("utf-8")
        seg_text = jieba.cut(text.replace("\t"," ").replace("\n"," "))
        outline = " ".join(seg_text)
        outline = outline.encode("utf-8") + "\t__label__" + e + "\n"
#         print outline
#         break

        if count < 10000:
            ftrain.write(outline)
            ftrain.flush()
            continue
        elif count  < 20000:
            ftest.write(outline)
            ftest.flush()
            continue
        else:
            break

ftrain.close()
ftest.close()


In [6]:


fr_label = open("type.tab","r")
lines_label = fr_label.readlines()
fr_label.close()
i = 0
with open("news.tab","r") as fr:    
    for line in fr:
#         print line
        outline = "__label__" + lines_label[i].rstrip() + "\t" + line
        i += 1
#         print outline
#         break
        if i <= 8000:
            ftrain.write(outline)
            ftrain.flush()
        else:
            ftest.write(outline)
            ftest.flush()
            
ftrain.close()
ftest.close()
        

In [80]:
# with open("stopwords.tab","r") as fr:
#     text = fr.read().rstrip().decode("utf-8")
# stoplist = set(text.split("\n"))
# print stoplist
def load_stopwords():
    f_stop = open('stopwords.tab')
    sw = [line.strip().decode("utf-8") for line in f_stop]
    f_stop.close()
    return sw
stop_words = load_stopwords()




dictionary = corpora.Dictionary(line.split() for line in open('news.tab'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid,docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()
dictionary.save('corpora.dict')
print 'ok'


2016-10-10 09:26:23,609 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2016-10-10 09:26:30,860 : INFO : adding document #10000 to Dictionary(105650 unique tokens: [u'\u4ee5\u8f6c', u'\u697c\u5170', u'\u65bd\u63f4', u'Chamerican', u'\u540e\u62a5']...)
2016-10-10 09:26:40,721 : INFO : adding document #20000 to Dictionary(186366 unique tokens: [u'\u94c5\u950c', u'\u540e\u62a5', u'\u7f16\u8bd1', u'\u864e\u4eba\u6b64', u'\u73b0\u5f79\u519b\u4eba']...)
2016-10-10 09:26:51,617 : INFO : adding document #30000 to Dictionary(254724 unique tokens: [u'\u751f\u5316\u5b66', u'\u94c5\u950c', u'\u9a6c\u52a0\u6da6', u'\u540e\u62a5', u'\u7f16\u8bd1']...)
2016-10-10 09:27:00,221 : INFO : adding document #40000 to Dictionary(314711 unique tokens: [u'\u8001\u53e3', u'\u751f\u5316\u5b66', u'\u94c5\u950c', u'\u9a6c\u52a0\u6da6', u'\u5c0f\u8c61\u822c']...)
2016-10-10 09:27:06,928 : INFO : adding document #50000 to Dictionary(358339 unique tokens: [u'\u9f88\u6c9f', u'360.00', u'\u9a6c\u52a0\u6d

ok


In [100]:
dictionary = corpora.Dictionary.load('corpora.dict')

2016-10-12 09:13:47,281 : INFO : loading Dictionary object from corpora.dict


In [5]:
# print "初始化停止词表-----"
# t_start = time.time()
# stop_words = load_stopwords()
# print "初始化停止词完成，用时%.3f秒" %(time.time() - t_start)

初始化停止词表-----
初始化停止词完成，用时0.002秒


In [7]:
class MyCorpus(object):
    def __iter__(self):
        for line in open("news.tab"):
            yield dictionary.doc2bow(line.split())

In [9]:
print "开始计算文本向量--"
fw = open("input.log","w")
t_start = time.time()

corpus_memory_friendly = MyCorpus()
corpus = []
num = 0
for vector in corpus_memory_friendly:
    fw.write(str(num) + "\n")
    fw.flush()
    num += 1
    corpus.append(vector)
    
# f = open('news.tab')
# texts = [[word for word in line.strip().split()] for line in f]
# texts = []
# count = 0
# for line in f:    
#     count += 1
#     fw.write(str(count) +"\n")
#     fw.flush()    
#     words = [word for word in line.strip().split()]
#     texts.append(words)
# fw.close()
print "计算文本向量完成，用时%.3f秒" % (time.time() - t_start)
fw.close()
# M = len(texts)
# print '文本数目：%d个' % M
# # print '正在建立词典--'
# # dictionary = corpora.Dictionay(texts)
# V = len(dictionary)
# # dictionary.save("test.dict")
print '正在保存文本向量--'
t_start = time.time()
# corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize("corpus.mm",corpus)
print "保存文本向量完成，用时%.3f秒" % (time.time() - t_start)

开始读入语料--
读入语料完成，用时84.376秒


In [15]:
corpus = corpora.MmCorpus("./corpus.mm")

In [27]:
print '正在计算文档TF-IDF --'
t_start = time.time()
corpus_tfidf = models.TfidfModel(corpus)[corpus]
corpus_tfidf.save('corpus_tfidf_model')

print "建立文档TF——IDF完成，用时%.3f秒" % (time.time() - t_start)

正在计算文档TF-IDF --
建立文档TF——IDF完成，用时108.745秒


/usr/local/lib/python2.7/dist-packages/gensim/interfaces.py:60: UserWarning: corpus.save() stores only the (tiny) iteration object; to serialize the actual corpus content, use e.g. MmCorpus.serialize(corpus)
  warnings.warn("corpus.save() stores only the (tiny) iteration object; "


In [8]:
#corpus_tfidf.load('corpus_tfidf_model')
corpus_tfidf = models.TfidfModel.load('corpus_tfidf_model')

2016-10-11 19:49:30,047 : INFO : loading TfidfModel object from corpus_tfidf_model
2016-10-11 19:49:30,132 : INFO : loading corpus recursively from corpus_tfidf_model.corpus.* with mmap=None
2016-10-11 19:49:30,132 : INFO : loading obj recursively from corpus_tfidf_model.obj.* with mmap=None


In [101]:
print 'LDA模型拟合推断--'
num_topics = 100
t_start = time.time()
lda = models.LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary,alpha='auto',eta='auto',
                      minimum_probability=0.0, update_every=1, chunksize=2000, passes=1)
lda.save('corpus_lda.model')
print 'LDA模型完成，训练时间为%.3f秒' % (time.time() - t_start)
#

2016-10-12 09:13:53,765 : INFO : using autotuned alpha, starting with [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
2016-10-12 09:13:53,767 : INFO : using autotuned eta, starting with [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.

LDA模型拟合推断--


2016-10-12 09:13:59,739 : INFO : running online LDA training, 100 topics, 1 passes over the supplied corpus of 121177 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2016-10-12 09:14:01,813 : INFO : PROGRESS: pass 0, at document #2000/121177
2016-10-12 09:14:07,904 : INFO : optimized alpha [0.0098722082515527495, 0.0098774199029652149, 0.0098516680919847471, 0.0098056635539345398, 0.0098465979076770693, 0.0098364752192325322, 0.0098210508620145062, 0.0098364967029591684, 0.0098364841128239025, 0.0098875517427644485, 0.0098312527744412372, 0.0098720180217684948, 0.0098210669337923107, 0.0097902362497473954, 0.0098158266220400459, 0.0098411489807106855, 0.0098311944477712087, 0.0098413717883423634, 0.009867108121035105, 0.009836350773081648, 0.0098465139414759505, 0.0098822537930532712, 0.0098313391503746603, 0.0098364184299264308, 0.0098363646717147552, 0.0098311923219393128, 0.00984

LDA模型完成，训练时间为1381.688秒


In [5]:
lda = models.LdaModel.load('corpus_lda.model')

2016-10-11 19:48:29,103 : INFO : loading LdaModel object from corpus_lda.model
2016-10-11 19:48:29,301 : INFO : loading id2word recursively from corpus_lda.model.id2word.* with mmap=None
2016-10-11 19:48:29,302 : INFO : loading expElogbeta from corpus_lda.model.expElogbeta.npy with mmap=None
2016-10-11 19:48:29,376 : INFO : setting ignored attribute state to None
2016-10-11 19:48:29,377 : INFO : setting ignored attribute dispatcher to None
2016-10-11 19:48:29,378 : INFO : loading LdaModel object from corpus_lda.model.state
2016-10-11 19:48:29,379 : INFO : loading sstats from corpus_lda.model.state.sstats.npy with mmap=None


In [64]:
#随机打印某10个文档的主题
M = 121177
num_show_topic = 10
print '10个文档的主题分布：'
doc_topics = lda.get_document_topics(corpus_tfidf)
idx = np.arange(M)
np.random.shuffle(idx)
idx = idx[:1]
for i in idx:
    topic = np.array(doc_topics[i])
    topic_distribute = np.array(topic[:,1])
#     topic_idx = topic_distribute.argsort()[:-num_show_topic-1:-1]
#     print ("第%d个文档的前%d个主题：" % (i, num_show_topic)), topic_idx
#     print topic_distribute[topic_idx]
    print topic_distribute
    print topic_distribute[[i for i in range(len(topic_distribute))]]
# num_show_term = 20
# for topic_id in range(20):
#     print '主题#%d:\t' % topic_id
#     term_distribute_all = lda.get_topic_terms(topicid=topic_id)
#     term_distribute = term_distribute_all[:num_show_term]
#     term_distribute = np.array(term_distribute)
#     term_id = term_distribute[:,0].astype(np.int)
#     print "词：\t",
#     for t in term_id:
#         print dictionary.id2token[t],
#     print "\n概率：\t",term_distribute[:,1]


10个文档的主题分布：
[ 0.06055677  0.00171167  0.00171167  0.00171167  0.00761395  0.00171167
  0.00171167  0.00171167  0.00171167  0.00381479  0.00171167  0.00171167
  0.00171167  0.00171167  0.00616209  0.00171167  0.00171167  0.00171167
  0.00171167  0.00171167  0.00171167  0.00171167  0.06757611  0.00171167
  0.00761395  0.01382995  0.00171167  0.00171167  0.00563034  0.00171167
  0.0274866   0.00171167  0.00171167  0.00171167  0.00171167  0.00171167
  0.00731906  0.00171167  0.00171167  0.00171167  0.00171167  0.00171167
  0.00807197  0.00171167  0.00171167  0.00853562  0.00655879  0.04370019
  0.00171167  0.00171167  0.00171167  0.00171167  0.00171167  0.00171167
  0.00171167  0.00171167  0.00171167  0.00171167  0.00171167  0.32393828
  0.00171167  0.01369325  0.00171167  0.00171167  0.01252028  0.00171167
  0.00171167  0.00171167  0.00171167  0.00171167  0.00171167  0.00171167
  0.00171167  0.00171167  0.00668308  0.01481375  0.08340348  0.00317652
  0.05330504  0.00499676  0.00171167  0

In [10]:
from sklearn import svm
from sklearn import datasets
import time

In [9]:
def show_accuracy(a,b,tip):
    acc = a.ravel() == b.ravel()
    print tip + '正确率： %.2f%%' % (100*float(acc.sum()) / a.size)

In [112]:
M = 121177
print '生成svm数据格式--'
fw = open('news_svm.tab','w')
with open('type.tab','r') as fr:
    lines = fr.readlines()
t_start = time.time()
doc_topics = lda.get_document_topics(corpus_tfidf)
# print doc_topics
idx = np.arange(M)
# np.random.shuffle(idx)
# idx = idx[:1000]
count = 0
for i in idx:
#     print i
    try:
        topic = np.array(doc_topics[i])
        topic_distribute = np.array(topic[:,1])
        distribute = [str(j) for j in topic_distribute]
#         print i ,len(distribute)
        if len(distribute) == 100:
            count += 1        
        outline = "\t".join(distribute) + "\t" + lines[i].strip() + "\n"
        fw.write(outline)
    except:
        continue
print len(idx),count
fw.close()
#     print topic_distribute[[i for i in range(len(topic_distribute))]]

print 'svm数据完成，时间为%.3f秒' % (time.time() - t_start)

生成svm数据格式--
121177 121177
svm数据完成，时间为326.500秒


In [12]:
print '生成svm模型--'
t_start = time.time()
data = np.loadtxt('news_svm.tab',dtype=np.float,delimiter='\t')
x,y = np.split(data,(-1,),axis=1)
# clf = svm.SVC(decision_function_shape='ovr')
clf = svm.SVC(C=1, kernel='rbf', gamma=0.001)
clf.fit(x,y)
print 'svm模型完成，时间为%.3f秒' % (time.time() - t_start)

y_hat = clf.predict(x)
show_accuracy(y,y_hat,'训练集')

生成svm模型--


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


svm模型完成，时间为5476.975秒
训练集正确率： 21.24%


In [6]:
from sklearn.externals import joblib
# joblib.dump(clf,'news_svm.model')
import numpy as np


data = np.loadtxt('news_svm.tab',dtype=np.float,delimiter='\t')
x,y = np.split(data,(-1,),axis=1)
print len(x[0])
print len(y[0])

clf = joblib.load('news_svm.model')
print clf.predict(x[0:4])
print y[0:4]

100
1
[ 11.   7.   8.   2.]
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
